In [18]:
import os
import pandas as pd
import plotly
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly as pys
import ipywidgets as widgets
from ipywidgets import Layout

import _global_scripts as gs

## Purpose

Consider removing the crt distance variable that discourages close trips and replace with something that discourages close AND encourages far distances

 - **2.9.1** 
 

## Inputs

In [20]:
# set whether you want PA or OD analysis
pa_od = 'PA'
pa_od_function = 'pa'

In [33]:
df_tdm_obs = pd.read_csv(f"_data/base_observed_summary_{pa_od_function}.csv").sort_values(by=['Source','station'], ascending=True).reset_index(drop=True)
if(pa_od == 'PA'):
        df_tdm_obs = df_tdm_obs.drop(columns={'Alt_PA','Alt_Direct_PA','Alt_Transfer_PA'})

# read in results for e2.9 after recalibration
path_brding_summary_m1 = r"_data/E2.9/M1_node.csv"
path_rider_summary_m1  = r"_data/E2.9/M1_link.csv"
path_brding_summary_291 = r"_data/E2.9/E2.9.1_node.csv"
path_rider_summary_291  = r"_data/E2.9/E2.9.1_link.csv"

In [34]:
# read in stations and summarize tdm results
df_stations1 = gs.df_stations[['station','N']]
df_tdm_m1 = gs.summarize_tdm_stats(path_brding_summary_m1,path_rider_summary_m1, df_stations1, 'TDM_M1', pa_od_function)
df_tdm_291 = gs.summarize_tdm_stats(path_brding_summary_291,path_rider_summary_291, df_stations1, 'TDM_2.9.1', pa_od_function)

# concat dataframes
df_tdm_obs_new = pd.concat([
        df_tdm_obs,
        df_tdm_m1,
        df_tdm_291
    ]).reset_index()
df_tdm_obs_new = df_tdm_obs_new.round(3).fillna(0)
df_tdm_obs_new = df_tdm_obs_new.drop(columns='index')
df_tdm_obs_new = df_tdm_obs_new.loc[:, ~df_tdm_obs_new.columns.str.contains('^Unnamed')]

# rename
sumStats = df_tdm_obs_new.copy()
sumStats = sumStats[sumStats['Source']!='TDM']
display(df_tdm_obs_new)

,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00
...,...,...,...,...,...,...
115,TDM_2.9.1,11-FARMINGTON STATION,walk,182.870,124.27,58.60
116,TDM_2.9.1,12-LAYTON STATION,walk,319.580,135.40,184.18
117,TDM_2.9.1,13-CLEARFIELD STATION,walk,577.690,446.52,131.17
118,TDM_2.9.1,14-ROY STATION,walk,307.190,283.22,23.97


In [35]:
# add a few more columns regarding percentage of boardings in relation to total boardings

# sum by source and station 
station_sum = sumStats.groupby(["Source", "station"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

# add All accessMode
station_sum["AccessMode"] = "All"
sumStats2 = pd.concat([sumStats, station_sum], ignore_index=True) 

accessmode_sum = sumStats2.groupby(["Source", "AccessMode"], as_index=False).agg({
    f"Brd_{pa_od}": "sum",
    f"Brd_Direct_{pa_od}": "sum",
    f"Brd_Transfer_{pa_od}": "sum"
})

accessmode_sum.rename(columns={
    f"Brd_{pa_od}": f"Source_Brd_{pa_od}", 
    f"Brd_Direct_{pa_od}": f"Source_Brd_Direct_{pa_od}", 
    f"Brd_Transfer_{pa_od}": f"Source_Brd_Transfer_{pa_od}"}, inplace=True)
sumStatsP = sumStats2.merge(accessmode_sum, on=["Source", "AccessMode"], how="left")

sumStatsP[f"Brd_{pa_od}_Perc"]          = sumStatsP[f"Brd_{pa_od}"] / sumStatsP[f"Source_Brd_{pa_od}"]
sumStatsP[f"Brd_Direct_{pa_od}_Perc"]   = sumStatsP[f"Brd_Direct_{pa_od}"] / sumStatsP[f"Source_Brd_Direct_{pa_od}"]
sumStatsP[f"Brd_Transfer_{pa_od}_Perc"] = sumStatsP[f"Brd_Transfer_{pa_od}"] / sumStatsP[f"Source_Brd_Transfer_{pa_od}"]
sumStatsP


,Source,station,AccessMode,Brd_PA,Brd_Direct_PA,Brd_Transfer_PA,Source_Brd_PA,Source_Brd_Direct_PA,Source_Brd_Transfer_PA,Brd_PA_Perc,Brd_Direct_PA_Perc,Brd_Transfer_PA_Perc
0,OBS,01-PROVO CENTRAL STATION,drive,2283.379,0.00,0.00,14955.118,0.00,0.0,0.152682,NaN,NaN
1,OBS,01-PROVO CENTRAL STATION,walk,945.908,0.00,0.00,5672.892,0.00,0.0,0.166742,NaN,NaN
2,OBS,02-OREM CENTRAL STATION,drive,1475.024,0.00,0.00,14955.118,0.00,0.0,0.098630,NaN,NaN
3,OBS,02-OREM CENTRAL STATION,walk,371.804,0.00,0.00,5672.892,0.00,0.0,0.065540,NaN,NaN
4,OBS,03-AMERICAN FORK STATION,drive,1423.768,0.00,0.00,14955.118,0.00,0.0,0.095203,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
130,TDM_M1,11-FARMINGTON STATION,All,815.360,759.86,55.50,15879.330,12888.03,2991.3,0.051347,0.058959,0.018554
131,TDM_M1,12-LAYTON STATION,All,1093.280,905.94,187.34,15879.330,12888.03,2991.3,0.068849,0.070293,0.062628
132,TDM_M1,13-CLEARFIELD STATION,All,1226.480,1104.21,122.27,15879.330,12888.03,2991.3,0.077238,0.085677,0.040875
133,TDM_M1,14-ROY STATION,All,1027.030,1003.65,23.38,15879.330,12888.03,2991.3,0.064677,0.077875,0.007816


In [36]:
def plotit(variable, access_mode):
    output.clear_output()  # Clear previous output before displaying new content
    global firstTime
    if firstTime:
    
        filtered_data = sumStatsP[sumStatsP['AccessMode'] == access_mode]
            
        # Create histogram
        fig = px.histogram(
            filtered_data, 
            x="station", 
            y=variable, 
            text_auto='.2s',
            color='Source', 
            barmode='group',
            height=400
        )
        fig.update_layout(
            xaxis_title="Station Name",
            yaxis_title=str(variable),
            legend_title="Model Version"
        )
        
        # Display the plot
        fig.show()
    
    else:
        firstTime = True

In [37]:
lstValues = list([
    f'Brd_{pa_od}',
    f'Brd_Direct_{pa_od}',
    f'Brd_Transfer_{pa_od}',
    f'Brd_{pa_od}_Perc',
    f'Brd_Direct_{pa_od}_Perc',
    f'Brd_Transfer_{pa_od}_Perc'])
accessModeOptions = ['drive', 'walk', 'All']

selectValues = widgets.Select(options=lstValues, value=(f'Brd_{pa_od}' ), description = 'Select Variable')
selectAccessMode = widgets.Dropdown(options=accessModeOptions, value='All', description='Access Mode')

# Set up a global variable to track whether the widgets have been changed
firstTime = False

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox = widgets.HBox([selectValues, selectAccessMode])

# create interactive widget
interactive_output = widgets.interactive_output(plotit, {'variable':selectValues, 'access_mode': selectAccessMode})

display(hbox)
display(interactive_output)
display(output)

Output()

Output()

## Conclusions

 - by reducing the number of access links from taz to station from 2 to 1, the number of boardings at end of line stations (provo and ogden) increased and the number of boardings at the second stations (orem and roy) decreased. This supports our conclusion form previous investigations that trips are bypassing end of line stations and boarding at second stations.
 - reducing the number of taz to station access from 2 to 1 could be part of the solution to fixing low boardings at provo station. However, it is important to still maintain number of boardings at orem station -- so definitily not the final solution.